In [ ]:
#Pkg.clone("https://github.com/bnels/PlotlyJS.jl.git")
Pkg.checkout("PlotlyJS","fix-jsstring")
using Eirene
using MAT
using CSV
using GraphPlot
using CSV
using LightGraphs
using DataFrames
using JLD
using Plots
using Measures
using StatPlots
using RCall
R"library(TDA)"
R"source('~/Dropbox/Top Sim and Homog/Scripts/bottleneck_computations_functions2.R')"


## Pre-defined functions

function toWeightedAdj_byweight(network_und,node_weights)
    #Assume that HIGHER node weight means born earlier-- this means we
    # want to keep the MINIMUM
    nNodes = length(node_weights)
    t0 = network_und.*node_weights
    t1 = deepcopy(t0)
    for i in collect(1:nNodes)
        for j in collect(i:nNodes)
            t1[i,j] = minimum([t0[i,j],t0[j,i]])
            t1[j,i] = t1[i,j]
        end
    end
    t1
end

# To calculate the bettiCurves from the barcodes
function bettiCurveFromBarcode(allPIs,nNodes,nmats,maxDim)
    

    bettiCurve = zeros(nmats,nNodes+1,maxDim)
    birthCurve = zeros(nmats,nNodes,maxDim)
    deathCurve = zeros(nmats,nNodes,maxDim)

    for dimn in collect(1:maxDim)
        for matn in collect(1:nmats)
            matn = Int(matn)
            currentCurve = allPIs[matn]
            currentCurveDim = currentCurve[dimn]
            for barn in collect(1:size(currentCurveDim,1))

                # Add to birth curve
                birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] = birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] +1


                if currentCurveDim[barn,2]>nNodes
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] +1
                    else 
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn]+1
                    deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] = deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] +1

                end
            end
        end
    end
    
    return bettiCurve, birthCurve, deathCurve 
end


## Calculating edge density
function calculateEdgeQuantity(jadj_array)
    nmats = size(jadj_array,3)
    nNodes = size(jadj_array,1)
    edgeQuantity = zeros(nmats,nNodes)
    
    # Loop through to compute number of edges at each filtration step
    for matn in collect(1:nmats)
        adj = jadj_array[:,:,matn];
        for noden in collect(1:nNodes)
            edgeQuantity[Int(matn),Int(noden)] = length(find(adj.=<noden))/2
        end
    end
    edgeQuantity = edgeQuantity.-35   # get rid of self-loops = 0
    
end

function calculateDegreesFiltration(jadj_array,nNodes)
    nmats = size(jadj_array,3)
    nNodes = size(jadj_array,1)
    degree_array = zeros(nNodes,nNodes,nmats)
    
    jadj_array[jadj_array.==(nNodes*2)] = 0
    
    
    # Loop through to compute number of edges at each filtration step
    for matn in collect(1:nmats)
        adj = jadj_array[:,:,matn];
        for noden in collect(1:nNodes)
            adj_i = deepcopy(adj)
            adj_i[adj_i.>noden] = 0
            adj_i[adj_i.>0] = 1
            degree_array[:,Int(noden),Int(matn)] = sum(adj_i,1)
        end
    end
    
    return degree_array
    
end


## Making weights from order
function orderToWeights(s_0_array,nNodes)
        s_wei_array = zeros(size(s_0_array))
    for i0 in collect(1:size(s_0_array,1))

        for n0 in collect(1:nNodes)
            n0 = Int(n0)
            s_wei_array[i0,n0] = indexin([n0],s_0_array[i0,:])[1]
        end
    end
    s_wei_array
end

function calculateTheoreticaMaxDistance(s_wei_array,n) 
    theo_dist= Vector{Float64}(n)   #10100 or 24160
    for i1 in collect(1:size(s_wei_array,1))
        norm1 = norm(s_wei_array[1,:].-s_wei_array[i1,:],Inf)
        theo_dist[i1] = norm1
    end
    return theo_dist
end

## Making Diagrams from barcode for TDA in R
function makeDiagramFromBarcode(barcode_array,graphn,maxDim,nNodes)

    diag = []
    infs = find(barcode_array[:,3].==(nNodes*2))
    barcode_array[infs,3] = nNodes+1

    for d in collect(1:maxDim)
        bd = barcode_array[graphn,d]
        infs = find(bd[:,2].==(nNodes*2))
        bd[infs,2] = nNodes+1
        nbars = size(bd,1)
        bda = hcat(d*ones(nbars),bd)
        diag = vcat(diag,bda)
    end
    diag
end




function computeBNDistances_glob(barcode_array)

    nReps = 101
    nGraphs = 100
    maxDim = 3

    graphOriginals = collect(1:101:10100)
    distanceBN_array = zeros(10100,3)

    for nG in graphOriginals
        diagO = makeDiagramFromBarcode(barcode_array,nG,maxDim,nNodes)
        println(nG)
        for nR in collect(1:(nReps-1))
            diagR = makeDiagramFromBarcode(barcode_array,(nG+nR),maxDim,nNodes)

            for nD in collect(1:3)
                distanceBN_dimn = R"computeBNDistance($diagO,$diagR,$nD)"
                distanceBN_array[(nG+nR),nD] = distanceBN_dimn
            end
        end
    end
    
    return distanceBN_array
end


function computeBNDistances_local(barcode_array)

    nReps = 2416
    nGraphs = 10
    maxDim = 3

    graphOriginals = collect(1:2416:24160)
    distanceBN_array = zeros(24160,3)

    for nG in graphOriginals
        diagO = makeDiagramFromBarcode(barcode_array,nG,maxDim,nNodes)
        println(nG)
        for nR in collect(1:(nReps-1))
            diagR = makeDiagramFromBarcode(barcode_array,(nG+nR),maxDim,nNodes)

            for nD in collect(1:3)
                distanceBN_dimn = R"computeBNDistance($diagO,$diagR,$nD)"
                distanceBN_array[(nG+nR),nD] = distanceBN_dimn
            end
        end
    end
    
    return distanceBN_array
end

In [ ]:
## Running PH

# Read in array from Matlab
filename = "NF_spatialGrowth_b1a100_1218_glob"

arr = matread("/Users/annsizemore/Dropbox/Top Sim and Homog/Results/$(filename)_forJul.mat")
print_with_color(:green,"Matrices loaded...")


arr1 = arr["jadj_array"]
nmats = size(arr1)[3]
nNodes = size(arr1)[1]
maxDim = 3

barcode_array = Array{Array{Float64}}(nmats, 3)

for i in collect(1:nmats)
    adj = arr1[:,:,i]    # Eirene reads from smallest to largest
 
    C = eirene(adj,maxdim = 3, record = "none",minrad = 0.5)

    for k = 1:maxDim
        # Record barcode. 
        barcode_array[i,k] = barcode(C,dim=k)
        end 
    
    
    if i%100 == 0
        print_with_color(:cyan,"\nRun $(i) finished.\n")
    end
    
    
    C = nothing
    gc()
    
end

print_with_color(:cyan,"Finished computing PH")
matwrite("/Users/annsizemore/Dropbox/Top Sim and Homog/Results/$(filename)_pis.mat",Dict("barcode_array" => barcode_array))
print_with_color(:blue,"\nDone :)")

In [ ]:
## Compute distances using TDA package in R (glob)
a = calculateDegreesFiltration(jadj_array,nNodes)
a[:,:,1]

In [ ]:
degree_array_mean

In [ ]:
## Processing output -- Betti Curves
## For Glob_reord
## Load everything

####### ---- stuff to define ------ #####
maxDim = 3
yLim = 70
graph_name = "NF_PropProb_d1_1218"
###### ---------------------------- ######

## Load Stuff
graph_name_0 = "$(graph_name)"
graph_name_glob = "$(graph_name)_glob"
graph_name_local = "$(graph_name)_local"


## Load just the reg stuff first --- the graph and 1000 reps
filename_0 = "$graph_name_0"
dict0 = matread("Results/$(filename_0)_pis.mat")
allPIs = dict0["allPersistenceIntervals"]
dict1 = matread("Results/$(filename_0).mat")
s_0_array = dict1["s_0_array"]
nReps = dict1["nReps"]
nGraphs = dict1["nGraphs"]
nNodes = dict1["nNodes"]
jadj_array = dict1["jadj_array"]
nmats = size(s_0_array,1)

## Clear
dict0 = nothing
dict1 = nothing
gc()


# Check we loaded the right stuff
println("Loaded $(graph_name_0) with $nGraphs graphs, $nReps replicates, and 
        $nNodes nodes")

## Make betti curve and other interesting things
bettiCurve, birthCurve, deathCurve = bettiCurveFromBarcode(allPIs,nNodes,nmats,maxDim)


s = squeeze(std(bettiCurve,1),1)
bettiCurve_mean = mean(bettiCurve,1)
bettiCurve_mean = squeeze(bettiCurve_mean,1)


### Plotting!!
gr()
adj = jadj_array[:,:,1]
adj[adj.==(nNodes*2)] = 0
adj[adj.>0] =1 

p1 = heatmap(adj,yflip = true,aspect_ratio=:equal, colorbar = false, color = :blues)
p2 = plot(1:(nNodes+1),bettiCurve_mean, label = ["B_1" "B_2" "B_3"], lw = 2,
    ribbon = s, size = (600,300),framestyle = :box,
    xlim = (0,nNodes),ylim=(0,yLim),size=(300,250))

## persistence diagrams
barcode1 = allPIs[1][1]
barcode1[barcode1.==(nNodes*2)] = nNodes

p3 = plot(1:nNodes,1:nNodes, c = RGB(0.55,0.55,0.55), framestyle =:box, aspect_ratio =:equal)
scatter!(barcode1[:,1],barcode1[:,2],markeralpha = 0.3,legend = false)

barcode2 = allPIs[1][2]
barcode2[barcode2.==(nNodes*2)] = nNodes
p4 = plot(1:nNodes,1:nNodes, c = RGB(0.55,0.55,0.55), framestyle =:box, aspect_ratio =:equal,
    ylim = (0,nNodes), xlim = (0,nNodes))
scatter!(barcode2[:,1],barcode2[:,2],markeralpha = 0.3,c = :blue,aspect_ratio = :equal,legend = false)


## We also need things like edge density, etc. 
edgeQuantityArray = calculateEdgeQuantity(jadj_array)
nEdgesFinal = binomial(Int(nNodes),2)
edgeDensityArray = edgeQuantityArray./nEdgesFinal
edgeDensityMean = squeeze(mean(edgeDensityArray,1),1)
edgeQuantityMean = squeeze(mean(edgeQuantityArray,1),1)
edgeDensityStd = squeeze(std(edgeDensityArray,1),1)
edgeQuantityStd = squeeze(std(edgeQuantityArray,1),1)

p5 = plot(1:nNodes,edgeDensityMean,ribbon = edgeDensityStd,legend = false)
plt = twinx()
plot!(plt,edgeQuantityMean,color=:green,ribbon = edgeQuantityStd,legend = false)

birthCurveMean = squeeze(mean(birthCurve,1),1)
birthCurveStd = squeeze(std(birthCurve,1),1)
deathCurveMean = squeeze(mean(deathCurve,1),1)
deathCurveStd = squeeze(std(deathCurve,1),1)

#p6 = plot(1:nNodes,birthCurveMean,c = [:blue :green :red],ribbon = birthCurveStd,label = ["Birth_1" "Birth_2" "Birth_3"])
#plot!(1:nNodes,deathCurveMean,c = [:blue :green :red], linestyle = :dot,ribbon = deathCurveStd,label = ["Death_1" "Death_2" "Death_3"],
#    framestyle = :box)
graphN = 1
p6 = plotBarcode(allPIs,nNodes,graphN,maxDim)

p7 = plotBarcode(allPIs,nNodes,graphN,maxDim)

degree_array = calculateDegreesFiltration(jadj_array,nNodes)
degree_array_mean = squeeze(mean(degree_array,3),3)

p8 = heatmap(degree_array_mean,yflip = true,aspect_ratio=:equal, colorbar = true, color = :viridis)


pall = plot(p1,p2,p3,p4,p5,p6,p7,p8,layout=(4,2),margin = 5mm,size= (800,900))
display(pall)
savefig("$(graph_name).pdf")

# Clear stuff
s_0_array = nothing
nReps = nothing
nGraphs = nothing
nNodes = nothing
#allPIs = nothing
gc()

In [ ]:
function plotBarcode(allPIs,nNodes,graphN,maxDim)


    counter1 = 0
    pbar = plot(1:6,zeros(6),c=:black)
    println("z")
    colors = [:blue :green :red]
    for dim in collect(1:maxDim)

        barn = allPIs[graphN][dim]
        barn = barn[sortperm(barn[:,1]),:]

        nbars = size(barn)[1]


        println("s")
        for cntr1 in collect(1:nbars)
            birth = barn[cntr1,1]
            death = barn[cntr1,2]

            plot!([birth, death],[cntr1+counter1, cntr1+counter1],c=colors[dim], legend = false)
        end

        display(pbar)



        counter1 = counter1+nbars
    end

    return pbar
end


In [ ]:
#### If necessary

pick = "old"
dmat_glob = []
if pick == "old"
    dict3 = matread("Results/$(graph_name_glob)_bottleneckDistances.mat")
    dmat_glob = dict3["distanceMat"]
else
    dmat_glob = computeBNdistances_glob(barcode_array)
end


In [ ]:
## Global reordering analyses
## Processing output -- Bottleneck Distnaces
# Requires processing in matlab and R


######## ----- ######
makeBox = 2
colors_orig = [RGB(0.2,0.3,0.6) RGB(0.23,0.6,0.3) RGB(0.6,0.2,0.4)]
colors_r = [RGB(0.3,0.5,1) RGB(0.23,0.9,0.4) RGB(0.9,0.3,0.5)]

######## ----- ######

#dict3 = matread("Results/$(graph_name_glob)_bottleneckDistances.mat")
#dmat_glob = dict3["distanceMat"]
#dmat_glob = distanceBN_array_glob
dict2 = matread("Results/$(graph_name_glob).mat")
s_0_array_glob = dict2["s_0_array"]
nNodes = dict2["nNodes"]
nGraphs = dict2["nGraphs"]
nReps = dict2["nReps"]
dict4 = matread("Results/$(graph_name_glob)_pis.mat")
allPIs_glob = dict4["allPersistenceIntervals"]
dict2 = nothing
dict3 = nothing
dict4 = nothing
gc()
print("loaded relevant data")
println(nGraphs)
println(nReps)


# Need to make an ordering to distance function
# Will make tomorrow for now will plot
s_wei_array_glob = orderToWeights(s_0_array_glob,nNodes)
theo_dist_glob = calculateTheoreticaMaxDistance(s_wei_array_glob,10100) 


inds1 = collect(0:nReps:(nReps*nGraphs-1))
inds1 = inds1 .+1
inds1 = Int.(inds1)
allPIs_orig = allPIs_glob[inds1]
indsr = setdiff(1:(nReps*nGraphs),inds1)
indsr = Int.(indsr)
allPIs_r = allPIs_glob[indsr]

## Make betti curve and such
bettiCurve_orig, birthCurve_orig, deathCurve_orig = bettiCurveFromBarcode(allPIs_orig,nNodes,nGraphs,maxDim)
bettiCurve_r, birthCurve_r, deathCurve_r = bettiCurveFromBarcode(allPIs_r,nNodes,length(allPIs_r),maxDim)
bettiCurve_orig_mean = squeeze(mean(bettiCurve_orig,1),1)
bettiCurve_r_mean = squeeze(mean(bettiCurve_r,1),1)
bettiCurveStd_orig = squeeze(std(bettiCurve_orig,1),1)
bettiCurveStd_r = squeeze(std(bettiCurve_r,1),1)


if makeBox == 1
    trace1 = PlotlyJS.box(;y=dmat_glob[:,1], x=theo_dist_glob)
    PlotlyJS.plot([trace1])
else
    #plotlyjs()
    gr()
    p1a = plot(0:nNodes,0:nNodes,c = RGB(0.64,0.64,0.64),title = "Distance plots")
    scatter!(theo_dist_glob,dmat_glob[:,1],title = graph_name, xlabel = "Theoretical max distance",
        ylabel = "Bottleneck Distance",aspect_ratio = :equal,)
    p1b = plot(1:(nNodes+1),bettiCurve_orig_mean, c=colors_orig, label = ["B_1" "B_2" "B_3"], linewidth = 2, ribbon = bettiCurveStd_orig)
    plot!(1:(nNodes+1),bettiCurve_r_mean,xlabel = "nNodes", ylabel = "Betti_k",title = graph_name,
        c = colors_r, linewidth = 2, ribbon = bettiCurveStd_r, label = ["B_1 reord" "B_2 reord" "B_3 reord"])
    p1all = plot(p1a,p1b,layout=(1,2),margin = 10mm,size= (800,400), legend=false)
    
    
end

display(p1all)
savefig("$(graph_name)_GlobReorder.pdf")      


allPIs = nothing
allPIs_orig = nothing
allPIs_r = nothing
gc()

In [ ]:
pick = "new"
dmat_local = []
if pick == "old"
    dict5 = matread("Results/$(graph_name_local)_bottleneckDistances.mat")
    dmat_local = dict5["distanceMat"]
else
    dmat_local = computeBNdistances_local(barcode_array)
end

In [ ]:
## Local reordering analyses
## Processing output -- Bottleneck Distnaces
# Requires processing in matlab and R


######## ----- ######
colors_orig = [RGB(0.2,0.3,0.6) RGB(0.23,0.6,0.3) RGB(0.6,0.2,0.4)]
colors_r = [RGB(0.3,0.5,1) RGB(0.23,0.9,0.4) RGB(0.9,0.3,0.5)]

######## ----- ######

#dict5 = matread("Results/$(graph_name_local)_bottleneckDistances.mat")
#dmat_local = dict5["distanceMat"]
dict6 = matread("Results/$(graph_name_local).mat")
s_0_array_local = dict6["s_0_array"]
nNodes = dict6["nNodes"]
nGraphs = dict6["nGraphs"]
nReps = 2146
dict7 = matread("Results/$(graph_name_local)_pis.mat")
allPIs_local = dict7["allPersistenceIntervals"]
dict5 = nothing
dict6 = nothing
dict7 = nothing
gc()
print("loaded relevant data")
println(nGraphs)
println(nReps)


# Need to make an ordering to distance function
# Will make tomorrow for now will plot
s_wei_array_local = orderToWeights(s_0_array_local,nNodes)
theo_dist_local = calculateTheoreticaMaxDistance(s_wei_array_local,24160) 


inds1 = collect(0:nReps:(nReps*nGraphs-1))
inds1 = inds1 .+1
inds1 = Int.(inds1)
allPIs_orig = allPIs_local[inds1]
indsr = setdiff(1:(nReps*nGraphs),inds1)
indsr = Int.(indsr)
allPIs_r = allPIs_local[indsr]

## Swap thing
theo_dist_swap = zeros(Int(nNodes),Int(nNodes),nGraphs)
bdist_swap = zeros(Int(nNodes),Int(nNodes),nGraphs)

iter = 1
for r in 1:nGraphs
    iter = iter+1
    
    for i0 in collect(1:nNodes)
        for j0 in collect((i0+1):nNodes)
            theo_dist_swap[Int(i0),Int(j0),Int(r)] = theo_dist_local[iter]
            theo_dist_swap[Int(j0),Int(i0),Int(r)] = theo_dist_swap[Int(i0),Int(j0),Int(r)]
            bdist_swap[Int(i0),Int(j0),Int(r)] = dmat_local[iter,1]
            bdist_swap[Int(j0),Int(i0),Int(r)] = bdist_swap[Int(i0),Int(j0),Int(r)]
            iter = iter+1
        end
    end
end

bdist_swap_mean = squeeze(mean(bdist_swap,3),3)
simRatio = bdist_swap_mean./theo_dist_swap[:,:,1]
println(size(bdist_swap_mean))

# Plot scatter of everything
gr()
p2a = plot(0:nNodes,0:nNodes,c = RGB(0.64,0.64,0.64),title = "Distance plots")
scatter!(theo_dist_local,dmat_local[:,1],title = graph_name, xlabel = "Theoretical max distance",
    ylabel = "Bottleneck Distance",aspect_ratio = :equal,)
p2b = heatmap(theo_dist_swap[:,:,1],yflip = true,aspect_ratio =:equal, title = "Theo Distance")
p2c = heatmap(bdist_swap_mean,yflip = true, aspect_ratio=:equal, title = "$(graph_name) BN Distance Dim1")
p2d = heatmap(simRatio,yflip = true, aspect_ratio=:equal, title = "Similarity Ratio Dim1")
p2all = plot(p2a,p2b,p2c,p2d,layout = (4),margin = 10mm, size = (900,800))
display(p2all)

savefig("$(graph_name)_local.pdf")

In [ ]:
# trace1 = PlotlyJS.box(;y=dmat_local[:,1], x=theo_dist_local)
# PlotlyJS.plot([trace1])


p9 = plot(0:nNodes,0:nNodes,c = RGB(0.64,0.64,0.64),title = "Distance plots", legend = false, aspect_ratio = :equal)
for i0 in collect(1:69)
#     print(i0)
    keep1 = dmat_local[theo_dist_local.==i0,1]
#     #print(theo_dist_local.==Int(i0))
    boxplot!([i0],keep1,linewidth = 0.5, markersize = 0.5, c = RGB(0.1,0.2,0.3))
end
display(p9)
#dmat_local[theo_dist_local.==5,1]
#boxplot(x = theo_dist_local, y = dmat_local[:,1])
#savefig(p::Union{Plot,SyncPlot}, "output_filename.pdf")
savefig("$(graph_name)_local_boxplots.pdf")

In [ ]:
s_0_array_local